### Online ML repositories
Sources: Kaggle, Google, Amazon, Microsoft, ML Repositories at UC Irvine and Carnegie Melon and LionBridge
<details><summary>Online ML datasets..EXPAND</summary>

*. https://towardsdatascience.com/top-sources-for-machine-learning-datasets-bb6d0dc3378b
1. Goole's Datasets Search Engine (https://datasetsearch.research.google.com/)
2. Kaggle  (https://www.kaggle.com/datasets)
3. Amazon  (https://registry.opendata.aws/)
4. UC Irvine ML Repository (https://archive.ics.uci.edu/ml/index.php)
>* https://archive.ics.uci.edu/ml/datasets.php
5. Visual Data on computer vision (https://www.visualdata.io/)
6. Carnegi Melon Datasets (https://guides.library.cmu.edu/machine-learning/datasets)
7. Microsoft Datasets (https://msropendata.com/)
8. Government Datasets (EU, US, NZ, India, Ireland)
9. Awesome Public Datasets at GitHub (https://github.com/awesomedata/awesome-public-datasets)
10. LionBridge Dataset (https://lionbridge.ai/datasets/)
4. https://elitedatascience.com/datasets

</details>

## Unsupervised ML Algorithms
* <b>Clustering:</b> K-Means Clutering

STEPS
* Load Data
* Prepare Data
    * null handling (na handling),
    * category indexing (StringIndexer) and binary vectoriation (OneHotEncoder)
    * vecor assembly transformation from input columns to 'features', the combined vector column (.transform)
    * scaling of combined 'features' column to 'scaledFeatures' using StandardScaler (.fit.transform)
        * Scaling is very useful when there is a big difference between features and scaledFeatures.
        * Scaling is also useful when fields vary in orders of large magnitudes, say one column expressed in thousands of miles and another column in milimeters, then scaling helps.
* Create KMeans model with 'scaledFeatures' and desired K value (matching with number of suspected clusters) in the constructor.
* Train this model with the vectorized scaled input dataframe and that is our final kmodel (.fit)
* kmodel.clusterCenters() gives an list of feature arrays, each array representing a centroid of a cluster
    * Each array represents a single point of n-dimension and is called a centroid.
    * The number of arrays will be same as value of k used, i.e. number of clusters.
    * The count of elements in each array is equal to the number of features.
* Run trained_model.transform(sdfScaledVectorFeatures) to get the result dataframe with predictions, where we can check the 'prediction' column. We can also check the 'prediction' column from the model (kmodel.summary.predictions.select('prediction'))
* We can evaluate the model using one of the following ways.
    1. kmodel.computeCost(sdfScaledVectorFeatures)
    2. ClusteringEvaluator('prediction', 'scaledFeatures').evaluate(kmodel.summary.predictions)

## Supervisd ML Algorithms
* <b>Regression:</b> LinearRegression
* <b>Classification:</b> LogisticRegression, DecisionTreeClsssifier, RandonForestClassifier, GBTClassifier

#### Machine Learning with various regressor and classifiers

<pre>
<details><summary>BUILD SESSION & LOAD DATA</summary>
###* <b>It is same for all models</b>
spark1 = SparkSession.builder.appName('lr_example').getOrCreate()
##all_data = spark1.read.format("libsvm").option("numFeatures", 10).load("sample_linear_regression_data.txt")
all_data = spark1.read.csv('Ecommerce_Customers.csv', inferSchema=True, header=True)

</details>
<details><summary>PREPARE DATA</summary>
<i>
###* <b>It is same for all models</b>
#* Use StringIndexer and optionally OneHotEncoder to convert categorical values into integers and then to binary vectors (optional)
#* For categorical columns use only the indices or binary vectorized category indices or binary vectors
#* Categorical text values need to be replaced with categorical indices using StringIndexer.
#* If there is an ordinal relationship between the values of a categorical field then the categorical indices can seve as a feature. E.g. different shades of a  color  or temperature such as Cold, Warm, and Hot.
#* If there is no relation in the values of the field, then we need to convert those categorical indices into binary vector where each value represents exactly single one value and rest zeroes, and this binary vector is more expressive for the ML algorithms to handle
#* The cateorical indices can be converted into binary vector using OneHotEncoder if no ordinal relation is detected between the different values of a specific categorical field.
#*   * e.g. temperature of 'Cold, 'Warm, 'Hot' are having ordinal relationship but a animal field with values 'dog'and 'cat' does not have any ordinal relationship.
#* Finally VectorAssembler is used to prepare the 'features' column for use with spark MLlib functions
</i>

assembler = VectorAssembler(inputCols=['col1','col2','ol3'], outputCol='features')
final_allfield_data = assembler.transform(all_data)   # This adds 'features' column to the result
final_data = final_allfield_data.select('features', 'my_label')  #* Spark ML algos need only these two columns
</code>

#* MLlib from spark needs only two columns 'features' and the label (the field to be predicted)
train_data, test_data = final_data.randomSplit([0.7, 0.3])    #Data is split into 70% (i.e. 0.7) for train_data and 30% for test_Data into two dataframes

</details>
<details><summary>TRAIN THE MODEL WITH TRAINING DATA</summary>
###* <b>It is similar, but</b> the constructors for different models differ in their parameters according to algorithm implementation.
<u>Create the models</u>
* linear=LinearRegression(labelCol='label', featuresCol='features', predictionCol='prediction')
* logistic=LogisticRegression(labelCol='label', featuresCol='features', predictionCol='prediction')
* dtc = DecisionTreeClassifier(labelCol='PrivateIndex',featuresCol='features')  ##* Here label is a categorical value which is indexed using StringIndexer and used in the tree algorithm
* rfc = RandomForestClassifier(labelCol='PrivateIndex',featuresCol='features')
* gbt = GBTClassifier(labelCol='PrivateIndex',featuresCol='features')

<u>Train the models</u>
trained_mdoel = model.fit(train_data)  #* Here 'model'  is one from linear / logistic / dtc / rfc / gbt
</details>
<details><summary>EVALUATE THE MODEL WITH TEST DATA (optional but helps improving the model by finetuning manually)</summary>
* test_results = trained_model.evaluate(test_data)  #* For Linear Regression and Logistic Regression use .evaluate()
* test_results = trained_model.transform (test_data) #* For tree based classifiers use .transform()
#* THIS EVALUATION helps us to finetune by changing model construction parameters and train_test_split porportion.
#* Test data evaluation result can be treated similar to trained_model.summary
#* the trained_model<b>.featureImportances</b> actually tells the relative importance of the various features of th etrained model that contributes maximum towardds the label prediction.

    <details><summary>What to check for evaluation</summary>
    * For Linear Regression check test_results.residuals DF, which shows a list of errors i.e. difference of predictions from the observations
    * For all classifiers check test_results.predictions DF, which shows the binary 'prediction' column (0/1) to be compared against the label field (0/1) along with two more fields rawPrediction and probability
        * This applies to Logistic Regression, DecisionTreeClassifier, RandomForestClassfier, GBTClassifier (GBT = GradientBoostedTree.
        * test_result.predictions is a dataframe with 5 fields [ 2 input fields (label, features) + 3 fields rawPrediction, probability, prediction (0 or 1)]
        * In "predictions" DF we need to compare prediction field against the label field to find the model performance.
    </details>
    <details><summary>Using Evaluators for classifiers</summary>
        * We can use BinaryClassificationEvaluator to find AUC (Area Under Curve [metricName=areaUnderROC or areaUnderPR] -- 0.5 means random guess, higher is better. ROC (Receiver Operator Curve) is default, Other is PR (Precision Recall curve)
        * We can use MulticlassClassifiactionEvaluator to find metrics f1(default), accuracy, weightedPrecision, weightedRecal
        * Most of these metrics are also available directly from the evaluation result "test_result".
        * I could not find for areaUnderPR and f1 attributes in test_result.
    </details>

</details>
<details><summary>DEPLOY THE MODEL (i.e. DO PREDICTION)</summary>
unlabeled_data = test_data.select('features')
predicted_data = trained_model.transform(unlabeled_data)
    
<details><summary>If the unlabeled data is a new data set</summary>
* If there is a separate dataset available for deployment, then once we are satisfied with the test_data results we can retrain the model with the entire pre-split data (i.e. train+test) and use this full_model to transform the deployment data.
##Use same model instance 'model' but 
#* use the full pre-split data set 'final_data' instead of train_data (70% of final_data)M
final_model = model.fit(final_data)

#* Prepare the 'features' column for the new unlabeled data
new_customers = spark.read.csv('new_customers.csv',inferSchema=True, header=True)
test_new_customers = assembler.transform(new_customers)

#* To Predict, the trained model needs only 'features' column, and rest of the fields are not utilised, so we need not remove these extra columns
final_predicted_result = final_model.transform(test_new_customers);
</details>
    
</details>
<details><summary>Using a PIPELINE (to prepare data, train a model and evaluate it)</summary>
We can optionally use pipeline (for simplicity) if there are a number of repeatitive tasks such as StringIndexer and OneHotEncoder  need to be applied for multiple categorical features in the input.

* my_pipeline = Pipeline(stages=[genderIndexer, embarkIndexer, genderEncoder, embarkEncoder, vecAssembler, myLogisticRegressionModel])

* Train the model with train data using
my_pipeline.fit(train_data)

* Evaluate the model by predicting test data using
my_pipeline.transform(test_data)

* pipeline.transform() uses the field 'prediction' not 'rawPrediction', so we need to explicitly tell Binary evaluator to use 'prediction column instead of default pick of rawPrediction field. This we can achieve by using rawPrediction = 'prediction' in binary evaluator constructor.
* So when using binary evaluator to evaluate the result from a pipeline we need to use rawPrediction='prediction' in binary evaluator contstructor.
* With MulticlassClassificationEvaluator default is 'prediction' column so no issues here.
</pre>

<details><summary><b>Linear Regression</b> - Some Theory...EXPAND</summary>
    
* Spark Machine Learning algorithms work only on two columns (features and label) for superised and one column (features) for unsupervised models
* Data_Transformations.ipynb
    * <b>StringIndexer : </b> converts catgorical values (or group names) found in a column into a numeric categorical index so that this categorical field can be used as an input feature.
    * <b>VectorAssembler : </b> combines multiple numeric,boolean, or vector input columns into a single vectorized feature column (default name is 'features')
    * Model training method ".fit()" uses this 'features' combined vectorized column.
    * the data needs to be in the form of two columns ("label","features") before Spark can accept the data!
</details>

<details><summary><b>Linear Regression</b> - Steps... EXPAND</summary>

<details><summary>Load input data</summary>
    
`    sdf_initial_data = spark1.read.csv('cruise_ship_info.csv',inferSchema=True,header=True)`
</details>
<details><summary>Prepare the data</summary>
    
    * Convert categorical values into categorical index values using StirngIndexer
    * Prepare a combined 'features' vectorized column (using VectorAssembler) holding all the numeric, boolean of other Vector.dense() types
    * The input dataframe for the training dataset need to be in the form of two columns ("label","features") for supervised model.
        * 'features' - It is the vector  column holding all the numeric, boolean or another vector types
        * 'label' - It is the column we want to predict for an unlabeled data set later.
    * Split the final input data into training_set and test_set
        * use sdf_final_data.randomSplit([0,7, 0.3]) to split the data into 70:30 percent proportaion.
        * each with two columns "label","features"
<b>CODE</b>
    <code>
    indexer = StringIndexer(inputCol="Cruise_line", outputCol="cruise_cat")
    indexed = indexer.fit(sdf_initial_data).transform(sdf_initial_data)
    assembler = VectorAssembler( inputCols=['Age',...], outputCol="features")
    output = assembler.transform(indexed)
    final_data = output.select("features", "crew")
    train_data,test_data = final_data.randomSplit([0.7,0.3])
    </code>
</details>
<details><summary>Train the model with the training data</summary>

* Train the model with the training data
    <code>
    lr = LinearRegression(labelCol='Yearly Amount Spent')   ## Create a Linear Regression Model object
    lrModel = lr<b>.fit</b>(train_data)   # Fit the model to the data and call this model lrModel
    </code>
</details>
<details><summary>Evaluate the performance of the trained model</summary>

* Evaluate the performance of the model by running it on the test data
    * `test_results = lrModel<b>.evaluate</b>(test_data)`
    * From the trained model (lrModel) check the coefficients and intercept of the trained model
    * From the evaluated test result check the tresiduals. We can ALSO check the residuals from the "summary" member of trained model (lrModel). Find the discrepancy between to two for your reference.
    * Check `test_results.residuals.show()` and `lrModel.summary.residuals.show()`
</details>
<details><summary>Run the model on unlabelled data</summary>

* Run the model on unlabvelled data (assuming production data)
    * Assumption is that machine must not have seen the production data on which it si going to be deployed, in a simple case test data is unseen by the model as it is not used in training the model
    * If we donot have separate unlabeled data
        * prepare unlabeled data by stripping off the labels from the test data
        * Transform() the unlabelled data using the available model
        * The result will have a 'prediction' column against for the 'features'.
        <code>
            * unlabeled_data = test_data.select('features')    #Remember only 'features' column in the unlabelled data
            * predicted_results = lrModel.transform(unlabeled_data)
        </code>
    * If we have separate unlabeled data, then we need to do somne extra work.
        * We need to use StringIndexer and VectorAssembler to prepare a dataframe with single vectorized 'features' column.
        * We can now create a new trained model using the entire initial input data without any train-test-split (as more data means better training).
        * Use transform() method of this "all-data" trained model to get the prediction result dataframe.
        * The result will have a 'prediction' column against for the 'features'.
</details>

</details>

<details><summary><b>Linear Regression</b> - A complete example</summary>

<pre>
<b><u>LOAD INPUT DATA</u></b>
from pyspark.sql import SparkSession
spark1 = SparkSession.builder.appName('cruise').getOrCreate()
df = spark1.read.csv('cruise_ship_info.csv',inferSchema=True,header=True)

<u>Check Loaded Data</u>
df.printSchema()
df.show()
df.describe().show()

##Dealing with the Cruise_line categorical variable
#Ship Name is a useless arbitrary string, but the cruise_line itself may be useful. Let's make it into a categorical variable!

df.groupBy('Cruise_line').count().show()

<b><u>PREPARE THE DATA</u></b> and <u>format for MLlib</u>
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="Cruise_line", outputCol="cruise_cat")
indexed = indexer.fit(df).transform(df)
indexed.head(5)

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
indexed.columns

assembler = VectorAssembler(
  inputCols=['Age',
             'Tonnage',
             'passengers',
             'length',
             'cabins',
             'passenger_density',
             'cruise_cat'],
    outputCol="features")

output = assembler.transform(indexed)

final_data = output.select("features", "crew")

<u>Test Train Split</u>
train_data,test_data = final_data.randomSplit([0.7,0.3])

<b><u>TRAIN THE MODEL WITH TRAINING DATA</u></b>
from pyspark.ml.regression import LinearRegression

#Create a Linear Regression Model object
lr = LinearRegression(labelCol='crew')

#Fit the model to the data and call this model lrModel
lrModel = lr.fit(train_data)

<b><u>EVALUATE THE PERFORMANCE OF THE TRAINED MODEL</u></b>
#Print the coefficients and intercept for linear regression
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

test_results = lrModel.evaluate(test_data)

print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))
print("R2: {}".format(test_results.r2))

#R2 of 0.86 is pretty good, let's check the data a little closer

<u>Find the correlation between the label('crew') and the predictors or features('passengers', 'cabins' etc) </u>
##If any of the predictors or features has high correelation, then the training may not be accurate.

from pyspark.sql.functions import corr
df.select(corr('crew','passengers')).show()
df.select(corr('crew','cabins')).show()

##Here we notice that there is a high correlation between our label (the 'crew' field) and some of the features such as cabins and the passenger count. So this prediction may not be accurate.

<b><u>PREDICT THE UNLABELLED (simulted) DATA USING THE MODEL</u></b>
unlabeled_data = test_data.select('features')    #Remember only 'features' column in the unlabelled data
predicted_results = lrModel.transform(unlabeled_data)

</pre>
</details>

###### Various Converters for data preparation
vector columns
* "StringIndexer"   [.fit().transform()]
    * This is used to convert categorical values into numeric categorical indices to be fed into the vector assembler input.
    * The resultant index column is used in input to vectorizer if the different values of the categorical field are related to each other.
* "OneHotEncoder"  [.transform()]
    * This is used to convert the numeric categorical indices into binary vector.
    * The resultant index column is used in input to vectorizer if the different values of the categorical field are not related.
* "VectorAssembler"  [.transform()]
    * This generates a vecor column (default named 'features') combining all the passed in numeric, boolean or other individual vectors
    * "StandardScaler"  [.fit().transform()]
    * This is used to scale the vectorized 'features' into 'scaledFeatures' for K-Means clustering algorithm
----------
* StringIndexer and OneHotEncoder transform a single column before vectorization
* StandardScaler scales the vectorized column.
* So if all are to be used at same time then below is the flow:
    * (String Indexer, OneHotEncoder)-->VectorAssembler-->StandardScaler

###### Spark DataFrames
<details><summary>Notes on Sparks Dataframes</summary>

Source: 
* chin_1_Spark DataFrame Basics.ipynb
* chin_1_Spark DataFrame Basics.ipynb
    * SQL operations on sdf
    * groupBy and aggregations
    * orderBy default ascending and descending
    * Handling missing data (na drop, replacing with fixed string  or with a computed value)
* chin_3_Spark DataFrame Exercise 9.1.ipynb
    * groupBy
    * changing data type of data frame column using sdf["col1"].cast('float') or sdf.selectExpr("cast(col1 as double") with selectExpr or 

<pre>
<details><summary>Loading a json file into a spark dataframe...EXPAND</summary>
    
* Use the .read() method from the spark session to create a DataFrame from a local file (.csv, .json etc)
my_json_people = "../Python-and-Spark-for-Big-Data-master/Spark_DataFrames/people.json"

from pyspark.sql import SparkSession
spark1 = SparkSession.builder.appName('Basics').getOrCreate()
sdf = spark1.read.json(my_json_people)

</details>
<details><summary>Changing the schema of a json file while reading into the dataframe...EXPAND</summary>

from pyspark.sql.types import StructField, IntegerType, StringType, StructType
data_schema = [StructField ('age', IntegerType(), False), 
                StructField('name', StringType(), True)]

final_type = StructType(fields=data_schema)
sdf2 = spark1.read.json(my_json_people, schema=final_type)

* Set a custom schema to the DataFrame through the sparkSession.read.json() operation
>* Use StructField, StructType and various base types IntegerType, StringType from pyspark.sql.types to change the schema of a json being imported into a spark DataFrame.
>* Use StructField(fieldName, StringType(), True) to define json type of a field of String type. For int type use IntegerType()
>* Use a list of StructField entries representing all the columns of the json file
>* Create the StructType schema (i.e. StructType(fields=list_of_StructFields)) and pass it as "schema" parameter to the sparksession read method (i.e. .read.json(jsonFile,schema=new_json_schema))
>* The DataFrame returned above will have specific data types instead of Strings. Validate it using sparkDF.printSchema()

</details>
<details><summary>Methods from my library tool "chinmay_tools.py"</summary>

* Make notebook shells enabled to print multiple results form a single shell
* Read and print a plain text (csv, txt, xml etc) file

* Convert the Spark DataFrame back to a pandas DataFrame using Arrow
result_pandasDF = sparkDF.select("*").toPandas()

* Get the call log xml data in a Pandas DataFrame
dfCallLogs = getCallLogXmlFromSuperbackup()   ## Calling from Chinmay_Utilities.ipynb uses xml.etree.ElementTree and pandas

* Create a Spark DataFrame from a pandas DataFrame using Arrow
sparkDF = tempSparkSession.createDataFrame(pandasDF)
sdfCallLogs = getSparkDFfromPandasDF(dfCallLogs)   ## Calling from Chinmay_Utilities.ipynb
</details>
<details><summary>Using RDBMS SQL Select statements on spark dataframes (session.sql())...EXPAND</summary>

#Register the spark dataframe as a table/view to be used like standard sql using sparkSession.sql()
sdfCallLogs.createOrReplaceTempView("call_logs")
#####Using pure SQL with DataFrames
* Register the dataframe as a table and
* Use sparkSession1.sql(SQL_STMT_table)spark1.sql("SELECT * FROM call_logs WHERE dur > 100 ORDER BY dur DESC").show()
* The where clause of this sql is not supporting LIKE clause
* This complex where clause (LIKE clause) is possible through direct "where" clause on spark dataframe (next statement)
</details>
<details><summary>SQL operations on dataframes (filter, select, where), computed column, renaming column etc...EXPAND </summary>

* Selecting / filtering data from DataFrame.
>* sparkSessn.sql(...) : Use direct Select SQL statements on the dataframe using sparkDF.sql(SQL_SELECT_STMT) e.g. "select col1, col2 from table1 where age > 60 and duration < 120"
>* sDF.select(...).where(...) : This returns subset of DataFrame(sDF) with specified columns(in select clause)  satisfying the condition passed is where clause.
>* sDF.filter(...) : This returns subset of DataFrame(sDF) with all columns satisfying the condition passed as filter e.g. "age > 18"

######Quick filtering: (one of the two alternate ways on sparkDF)
######sparkDF.select(*).where(my_condition)
######sparkDF.filter(my_condition)

my_where_clause = "upper(name) like '%SEEC%QA%' AND dur > 50"

#Querying the Spark DataFrame directly with WHERE clause
#This sort of complex where clauses (e.g. LIKE clauses) are not possible with pandas dataframe

#sdfCallLogs.select("*").where("upper(name) like '%SEEC%QA%'").show()

#Convert the filtered data into pandas data frame which can be processed or outputted into a file
df_qa = sdfCallLogs.select("*").where(my_where_clause).toPandas()
sdfCallLogs.filter(my_where_clause).toPandas()
#Rename the columns before converting to Pandas DataFrame
df_qa2 = sdfCallLogs.filter(my_where_clause).select('name', 'number', 'dur', 'time') \
    .withColumnRenamed('dur', 'Duration (Sec)').withColumnRenamed('name', 'Name')\
    .withColumnRenamed('time', 'Date').withColumnRenamed('number', 'Phone Number').toPandas()

#Below line inserts a new column with value double of the current 'dur' columns value
df_qa2B = sdfCallLogs.filter(my_where_clause).select('name', 'number', 'dur', 'time') \
            .withColumn('double_duration',sdfCallLogs['dur']*2).toPandas()

#To show a pd.DataFrame without column index
df_qa2.style.hide_index()
df_qa2B.style.hide_index()

#To write an dataframe to an excel file without index column
df_qa2.to_excel('1.xlsx',index=False)

sdfCallLogs.filter(my_where_clause).filter((sdfCallLogs['dur']>300) & ~(sdfCallLogs['dur']<1000)).show()
sdfCallLogs.filter(my_where_clause).filter('dur>300 and dur>=1000').show()  ## Filters can be cascaded

###My Library methods
######Convert the Spark DataFrame into Json
######Convert a Pandas DataFrame into Json by first converting into a Spark DataFrame
getJsonFromSparkDF(sdfCallLogs)   ## Calling from Chinmay_Utilities.ipynb
#To get Json from a pandas DataFrame fist convert itinto a Spark DataFrame and then get Json from it
getJsonFromSparkDF(getSparkDFfromPandasDF(dfCallLogs))   ## Calling from Chinmay_Utilities.ipynb

#####Check types of DataFrame, Columns and displaying selected columns as DataFrame
sdf.select('col1', 'col2').show() --> returns one or more columns as a dataframe --> Pass one column to select clause to get that column as a DF
sdf.select('Age').show()   # This returns a dataframe of selected columns
sdf[sdf['Age']>0].show()    # This works similar to regular pandas dataframe filtering
sdf1 = sdf.withColumnRenamed('age', 'old_age') ## Renames the column 'age' as 'old_age'
sdf2 = sdf.withColumn('double_age',sdf['age']*2)
* Inserts a computed column named 'double_age' contaning comutation result from one or more other columns
* If the name supplied matches with name of an existing column then that column is renamed with new data ? - TO VALIDATE
</details>
<details><summary>GroupBy and aggregates in a sparks dataframe, Handling missing data (<b>chin_2_Spark DataFrame Basics-withGroupBy.ipynb</b>)</summary>

* Use "groupBy('col1')" to group the data by the specified column in a DataFrame and run aggregate functions on the groupby object.
* Multiple aggregations can be done with one aggregation per column
sdf_stock.groupBy('Year').agg({'Price':'max', 'Close':'avg', 'Open':'min', 'High':'max', 'Low':'min'})\
                .select('Year', \
                    format_number('max(Price)', 2).alias('max_price'), \
                    format_number('avg(Close)', 2).alias('avg_close'), \
                    format_number('min(Open)', 2).alias('min_open'), \
                    format_number('max(High)', 2).alias('max_high'), \
                    format_number('min(Low)', 2).alias('min_low'))
sdf_stock.orderBy('Year').show()

* Handling missing values

#sdf_missing.na.fill(999).na.fill('CHINMAY').show()
sdf_missing.na.fill('ANAND', subset='Name').show()
sdf_missing.na.fill('ANAND', ['Name', 'Sales']).show()  # integer column is ignored as passed value is of string type
* We can fill the na values using "fillna()" method from pandas dataframe using a constant value or with 'ffill' for forward fill and 'ackfill' for backward fill from the nearest non-null value of tjhe field.

from pyspark.sql.functions import mean, avg
mean_sales = sdf_missing.select(mean(sdf_missing['Sales'])).collect()[0][0]
sdf_missing.na.fill(mean_sales, ['Sales']).show()
</details>
</pre>

</details>

<details><summary><b>Applying multiple aggregates on a groupby object for a datafrrame of multiple numeric columns</b></summary>
    
* We can apply multiple aggregates to a groupby object of a DataFrame with one aggregate per each numeric column of the DataFrame
* Syntax:
>* grpByYr.agg({'colNm1':'aggFn1','colNm2':'aggFn2', 'colNm3':'aggFn3'}).show()
>* grpByYr.agg({'Open':'max','Close':'min', 'Volume':'avg'}).show()
* If a specific column is repeated for more than one aggregate functions, then the last aggregate function is operated for the column, all peevious ones on the same column are ignored
</details>

### REGRESSION EVALUATION

* Regression models like Linear Regression predict continuous values
* Categorical values are Classification models which categorize the items into distinct categories
* Metrics like "accuracy" or "recall" are not useful for regression problems
* Evaluation metrics for Regression are designed for continuous values, some of these are:
>* Mean Absolute Error ($\frac{1}{n} \sum_{i=1}^n |y_i-\hat{y_i}|$)
>* Mean Squared Error ($\frac{1}{n} \sum_{i=1}^n (y_i-\hat{y_i})^2$)
>* Root Mean Squared Error ($\sqrt{\frac{1}{n} \sum_{i=1}^n (y_i-\hat{y_i})^2}$)
>* R Squared Values
>> R square (or .r2) is basically statistical property of our model, it is not a evaluation matrix.

#### Mean Absolute Error (MAE):

* ### $\frac{1}{n} \sum_{i=1}^n |y_i-\hat{y_i}|$
* This is the mean of the absolute value of errors i.e. just the average error
* Example: If we are predicting the price of a house based on its features then MAE says on average how far we are off on hte hous price.
* N.B. The error is the difference between the predicted label ($\hat{y_i}$), which is on the regression line) and the actual label ($y_i$) for a value of predictor x i.e. on the same vertical line. If the actual point falls below the regression line then this difference will be negateive and we cake the absolute value as the negative ones will cancel out the positive values giving wrong result. Hence we take the absolute values.

#### Mean Squared Error (MSE):

* ### $\frac{1}{n} \sum_{i=1}^n (y_i-\hat{y_i})^2$
* This is the mean of the squared errors i.e. just the average error
* Being squared, larger errors are noted more than with MAE. Hence MSE is more popular than MAE.
* If the error i.e difference between actual value($y_i$) and predicted value ($\hat{y_i}$) is more, then the difference is punished more and we take the average we can see that error more. MSE is more popu;lar as ti takes into account when we are off by very large amount.
* The issue will be in unit, which is a squared unit here, so we take suare root of MSE  to get RMSE (Root Mean Absolute Error)
* N.B. The error is the difference between the predicted label ($\hat{y_i}$), which is on the regression line) and the actual label ($y_i$) for a value of predictor x i.e. on the same vertical line.

#### Root Mean Absolute Error (RMSE):

* ### $\sqrt{\frac{1}{n} \sum_{i=1}^n (y_i-\hat{y_i})^2}$
* This is root of MSE i.e. root of the mean of the squared errors
It is most popular as its unit is same as that of y, and at the same time errors are punishedd more for higher difference.
* Being squared, larger errors are punished more than with MAE. Hence MSE is more popular than MAE.
* If the error i.e difference between actual value($y_i$) and predicted value ($\hat{y_i}$) is more, then the difference is punished more and we take the average we can see that error more. MSE is more popu;lar as ti takes into account when we are off by very large amount.
* N.B. The error is the difference between the predicted label ($\hat{y_i}$), which is on the regression line) and the actual label ($y_i$) for a value of predictor x i.e. on the same vertical line.

#### R Squared Values (R2)

* R squared values are also known as "Coefficient of determination"
* It is not an error metric, more of a statistical measure of our regression model.
    * It is the proportion of the variance in the dependent variable ($y_i$) that is predictable from the independent variable(s), i.e. $x1_i, x2_i etc$
* This tells how much variance our model accounts for.
* Its values lie between 0 and 1 (i.e between 0% to 100%)
* Example: if the value is 0.9 means it explains 90% of the variance of the data.
* Adjusted R squared value is a different way of obtaininng $R^2$
    * During machine learning if we get a negative value for $R^2$, then it is calculated using a different method.
* When we want to compare models to each other, we should use Adjsuted $R^2$ not a normal one.
    * Simple numerical comparison fo $R*2$ is nto enough a thorough analysis is needed.
* We can use this to understand our model better or compare models but it is not the sole source of evaluating a model.

# NEXT: Linear Regression Example Code Along

#### Example: 
* ###### Here we are trying to predict a customer's Total Amount Expenditure (a continuous money value)
* ###### We will then convert the realistic data into a format acceptable by Spark's MLlib.